pip install requests

In [1]:
import requests
from requests.auth import HTTPBasicAuth
import datetime

import numpy as np
import pandas as pd
import json

import warnings
#warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize

## Скрипт для обращения к Omnidesk API

In [2]:
class OmnideskAPI:
    def __init__(self, sub_domain, domain, email, api_key):
        self.host = 'https://' + sub_domain + '.' + domain
        self.auth = HTTPBasicAuth(email, api_key)

    def send_request(self, method, endpoint, case_id=None,  params=None):
        """
        method: get или post
        endpoint: cases, filters, case, staff, messages
        case_id=None: Испльзуется в case или messages
        """
        ENDPOINTS = {
            'cases': 'cases.json',
            'filters': 'filters.json',
            'case': 'cases/' + str(id) + '.json',
            'staff': 'staff.json',
            'messages': 'cases/' + str(case_id) + '/messages.json'
        }
        #print(self.host + ENDPOINTS[endpoint])
        if method == 'get':
            result = requests.get(url=self.host + ENDPOINTS[endpoint], params=params, auth=self.auth).json()
        elif method == 'post':
            result = requests.post(self.host + ENDPOINTS[endpoint], params=params, auth=self.auth).json()
        else:
            result = ''
        return result

## Параметры для API

In [3]:
omnidesk_config = {
    "sub_domain": "vudeba10",
    "domain": "omnidesk.ru/api/",
    "email": "myuferov@gmail.com",
    "api_key": "eebb542a4fb257eebc2779d0d"
}
params = {
    'from_time' : datetime.date(2021,5,27)
}

## Получение списка обращений и преобразование его в Dataframe 

In [4]:
omnidesk = OmnideskAPI(**omnidesk_config)
staff = omnidesk.send_request("get", "cases", params=params)

data_case = pd.DataFrame()
for i in range(0, staff['total_count']):
    data_case=pd.concat([data_case, json_normalize(staff[str(i)]['case'])] , ignore_index=True)
data_case=data_case[['case_id','user_id','staff_id','created_at']]
data_case_id=data_case.case_id.unique()

data_case
#staff

<ipython-input-4-ea21103c7ccf>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_case=pd.concat([data_case, json_normalize(staff[str(i)]['case'])] , ignore_index=True)


,case_id,user_id,staff_id,created_at
0,169278691,23351208,29386,"Thu, 27 May 2021 12:20:22 +0300"
1,169278689,23351208,29386,"Thu, 27 May 2021 12:20:21 +0300"
2,169278687,23351208,29386,"Thu, 27 May 2021 12:20:20 +0300"
3,169278686,23351208,29386,"Thu, 27 May 2021 12:20:01 +0300"


## Просмотр ответов и преобразование их в Dataframe

In [5]:
data_message = pd.DataFrame()
s=[]
for i in data_case_id:
    staff1 = omnidesk.send_request("get", "messages",case_id=i)
    for j in range(0, staff1['total_count']):
        data_message=pd.concat([data_message, json_normalize(staff1[str(j)]['message'])] , ignore_index=True)
        s.append(i)
data_message=data_message[['message_id','content','content_html']]
data_message['case_id']=s

data_message

<ipython-input-5-4e18898d0c06>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data_message=pd.concat([data_message, json_normalize(staff1[str(j)]['message'])] , ignore_index=True)


,message_id,content,content_html,case_id
0,0,Your knowledgebase is currently filled with sa...,,169278691
1,0,Connect your company's Facebook page to suppor...,,169278689
2,0,Connect your company's Twitter account to supp...,,169278687
3,232064593,Здравствуйте!\n\nРад приветствовать вас на Омн...,,169278686


## Объединение Dataframe списка обращений и ответов 

In [6]:
data=data_case.set_index('case_id').join(data_message.set_index('case_id'))

#замена пропусков NAN значениями. И удаление столбцов 'content' или 'content_html' если они полностью пустые
data.replace('', np.nan, inplace=True)
data.dropna(axis=1,how='all', inplace=True)

data

,user_id,staff_id,created_at,message_id,content
case_id,,,,,
169278691,23351208,29386,"Thu, 27 May 2021 12:20:22 +0300",0,Your knowledgebase is currently filled with sa...
169278689,23351208,29386,"Thu, 27 May 2021 12:20:21 +0300",0,Connect your company's Facebook page to suppor...
169278687,23351208,29386,"Thu, 27 May 2021 12:20:20 +0300",0,Connect your company's Twitter account to supp...
169278686,23351208,29386,"Thu, 27 May 2021 12:20:01 +0300",232064593,Здравствуйте!\n\nРад приветствовать вас на Омн...
